# Arabic Fraud Detection

This notebook demonstrates the steps to:
1. Load and preprocess the Arabic ride-hailing dataset.
2. Fine-tune a pre-trained Arabic language model (`AraBERT`) for fraud detection.
3. Evaluate the model's performance.

In [ ]:
# Install dependencies
!pip install pandas scikit-learn transformers datasets

## Step 1: Load and Explore the Dataset

In [ ]:
import pandas as pd

# Load the dataset
file_path = "../data/df_rating&comment.xlsx"
df = pd.read_excel(file_path)
df.head()

,id,comment
0,1,اصلا ماجاني وضليت انتظر فترة طويلة وراها اجرت ...
1,2,NaN
2,3,NaN
3,4,الرحله لغاها بدون سبب وتعامل سيئ
4,5,لم ياتي السايق و اخذ الاجرة


## Step 2: Preprocess the Data

In [3]:
word_list = [
    "تركني", "لا يأتي", "لم أكن", "لم ياتي", "لم يحضر", "لم يصل",
    "لن ياخذني", "ما اجا", "ما اجاني", "ما اجة", "ما اجه", "ما اجى",
    "ما اخذني", "ما حضر", "ما وصلني", "مااجاني", "ماجاني", "ماوصل", "موصل"
]

def label_fraud(comment):
    if not isinstance(comment, str):
        return 0
    for word in word_list:
        if word in comment:
            return 1
    return 0

# Apply preprocessing
df['label'] = df['comment'].apply(label_fraud)
df = df.dropna(subset=['comment'])
df.head()

,id,comment,label
0,1,اصلا ماجاني وضليت انتظر فترة طويلة وراها اجرت ...,1
3,4,الرحله لغاها بدون سبب وتعامل سيئ,0
4,5,لم ياتي السايق و اخذ الاجرة,1
5,6,السائق لم ياتي واستقطع مبلغ الرحلة,1
7,8,اكثر من ربع ساعة انتظر الكابتن على النقطة ولم ...,1


## Step 3: Fine-Tune the Model

In [4]:
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Split dataset into train and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['comment'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# Tokenize the data
model_name = "aubmindlab/bert-base-arabertv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Convert to PyTorch dataset
class FraudDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = FraudDataset(train_encodings, train_labels)
val_dataset = FraudDataset(val_encodings, val_labels)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

# Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3644729/643939653.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0

Epoch,Training Loss,Validation Loss
1,No log,0.578309
2,No log,0.560594
3,No log,0.564567


TrainOutput(global_step=9, training_loss=0.6192479663425021, metrics={'train_runtime': 40.062, 'train_samples_per_second': 2.696, 'train_steps_per_second': 0.225, 'total_flos': 2885999388480.0, 'train_loss': 0.6192479663425021, 'epoch': 3.0})

## Step 4: Evaluate the Model

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Get predictions
predictions = trainer.predict(val_dataset)
preds = predictions.predictions.argmax(-1)

# Calculate metrics
accuracy = accuracy_score(val_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(val_labels, preds, average="binary")

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.78
Precision: 0.78
Recall: 1.00
F1 Score: 0.88


## Conclusion

The model was fine-tuned successfully, achieving a good balance between precision, recall, and F1-score. Further improvements could involve hyperparameter tuning or augmenting the dataset.